In [ ]:
import pandas as pd
import numpy as np
import os
import json, requests
import time
import sqlite3

In [ ]:
city_country = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/city_country_separate.csv')

In [ ]:
city_country.iloc[0:20, :]
city_country.iloc[20:40, :]
city_country.iloc[40:60, :]
city_country.iloc[60:80, :]
city_country.iloc[80:100, :]
city_country.iloc[100:120, :]
city_country.iloc[120:139, :]

In [ ]:
she = city_country.iloc[12, :]


In [ ]:
test = city_country.iloc[[2,12,16,69], :].copy()


In [ ]:
test

In [ ]:
key = os.getenv('G_API_KEY')
#base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'
#maxprice = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}&query={}&minprice={}&maxprice={}'

In [ ]:
query_dict = {'attractions': "{}+points+of+interest",
              'restaurants': "best+restaurants+in+{}",
             'restaurants_one': 'best+restaurants+in+{}&minprice=1&maxprice=1',
             'restaurants_two': 'best+restaurants+in+{}&minprice=2&maxprice=2',
             'restaurants_three': 'best+restaurants+in+{}&minprice=3&maxprice=3',
             'restaurants_four': 'best+restaurants+in+{}&minprice=4&maxprice=4'} #build seperate query for min/max price 
city_list = city_country.iloc[4:6, :].copy()


In [ ]:
query3 = query_dict['attractions']
#url_price = base_url.format(key) + '&query={}'.format(query3.format('portland'))

In [ ]:
#utils.py
def format_city(city_list):
    city_list.replace(' ', '+', regex=True, inplace=True)
    return city_list

In [ ]:
city_list = format_city(city_list)

In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'

def city_query_selection(city_df, query_dict):
    for query in query_dict:
        for city in city_df.city:
            country = city_df.loc[city_df['city'] == city, 'country'].item()
            id = city_df.loc[city_df['city'] == city, 'id'].item()
            table_name = find_table_name(query_dict, query_dict[query])
            result = check_db(table_name, city)
            if result:
                print('inside if ')
                continue
            else:
                print('inside else')
                print(city, country, query_dict[query])
                pipeline(city, query_dict[query], country, id, query_dict)
                

    return 


# Pipeline - main.py
def pipeline(city, query, country, id, query_dict):
    url = build_url(city, query, country)
    data = find_places_api(url)
    table_name = find_table_name(query_dict, query)
    json = check_json(data)
    df = create_df(json, city, country, id, table_name)
    write_db(table_name, df)
    
    return df


def build_url(city, query, country):
    base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'
    cc = [city, country]
    cc = '+'.join(cc)
    url = base_url.format(key) + '&query={}'.format(query.format(cc))
    return url


def make_next_page_url(results, base_url):
    
    return base_url.format(key) + '&pagetoken={}'.format(results['next_page_token'])


def find_places_api(url, data=[]):
    results = requests.get(url).json()
    time.sleep(7)
    data = data + results['results']
    print(results.keys())
    print(results['status'])
    status = results['status']
    #if status == 'ZERO_RESULTS':
        # Send back to city_query_selection
    if 'next_page_token' not in results.keys() and results['status']=='OK':
        print('no next')
        return data
    elif results['status']=='OK':
        print('next')
        new_url = make_next_page_url(results, base_url=base_url)
        return find_places_api(new_url, data)
    else:
        return data 



def create_df(json_data, city, country, id, table_name):
    df = pd.json_normalize(json_data)
    df['city'] = city.replace('+', ' ')
    df['country'] = country.replace('+', ' ')
    df['id'] = id
    df = column_selection(df, table_name)
    return df


def column_selection(df, table_name):

    cols_to_keep = ['country', 'city'] + cols_to_check + ['id']
    df = df[cols_to_keep].copy()
    df.rename(columns={'formatted_address': 'address',
                       'geometry.location.lat': 'latitude',
                       'geometry.location.lng': 'longitude'}, inplace=True)
    df['types'] = df.types.astype(str)
    
    return df
    
# Utils.py    
def find_table_name(query_dict, query):
    table_name = (list(query_dict.keys())[list(query_dict.values()).index(query)])
    
    return table_name

#db_connect.py
def write_db(table_name, df):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))

    try:
        df.to_sql(table_name, con=conn, if_exists="append", index=False)
        
    except sqlite3.DatabaseError as er:
        print('er:', er.message)

    return 


def check_db(table_name, city):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))

    cur = conn.cursor()
    try: 
        cur.execute("""SELECT city from {} where city = '{}'""".format(table_name, city))
        result = cur.fetchone()
    except sqlite3.DatabaseError as er:
        print(er)
        result = []
        
    return result
# append data or delete and recreate when rerunning
# error handling if write to db doesn't suceed report error back. 

In [ ]:
cols_to_check = ['name', 'formatted_address', 'price_level',
                'rating', 'user_ratings_total', 'types', 'place_id']
def check_json(json_data):

    for col in cols_to_check:
        try:
            json_data[0][col]

        except Exception as e:
            json_data[0][col] = np.nan

    return json_data

    


In [ ]:
    cols_to_check = ['name', 'formatted_address', 'price_level',
                'rating', 'user_ratings_total', 'types', 'place_id']
    

In [ ]:
cols_to_keep

In [ ]:
check_db('attractions','thailand')

In [ ]:
(list(query_dict.keys())[list(query_dict.values()).index(q)])

In [ ]:
query_dict['restaurants']

In [ ]:
huh = city_query_selection(test, query_dict)

In [ ]:
test

In [ ]:
type(huh)

In [ ]:
turkey.city

In [ ]:
a = 'attractions'
cit = she.city
q = 'best+restaurants+in+{}&minprice=1&maxprice=1'
country = she.country
table = 'restaurants_one'


In [ ]:
url = build_url(cit, q, country)

In [ ]:
url

In [ ]:
data = find_places_api(url)
data

In [ ]:
d = check_json(data)

In [ ]:
d[0]

In [ ]:
table_name = find_table_name(query_dict, q)
print(table_name)

In [ ]:
id = 9
df0 = create_df(d, c, country, id, table)


In [ ]:
id

In [ ]:
print(q)

In [ ]:
df0

In [ ]:
write_db(table_name, df2)

In [ ]:
df2.dtypes

In [ ]:
#df['country'] = df.country.astype('string')
#df['city'] = df.city.astype('string')
#df['name'] = df.name.astype('string')
#df['address'] = df.address.astype('string')
#df['id'] = df.id.astype('int')
#df['place_id'] = df['place_id'].astype('string')
df2['types'] = df2.types.astype(str)

In [ ]:
df

In [ ]:
def retrieve_city():
    try:
        conn = sqlite3.connect('test.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
        
    city_df = pd.read_sql_query("select * from city_country limit 5;", conn)
    
    return city_df

In [ ]:
dfff = retrieve_city()

In [ ]:
dfff